### Objective: 
In this notebook, I am testing out the AbodyBuilder2 model from the ImmuneBuilder paper. 
To get it to work, I needed to start a new environment. I also downloaded the weights manually as it time out (at work). 


#### Import some packages

In [1]:
import pandas as pd
import nglview as ngl
from ImmuneBuilder import  ABodyBuilder2
from pathlib  import Path
import matplotlib.pyplot as plt
from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO
from tqdm.notebook import tqdm


#### Set up paths to an antibody. 

In [13]:
desktop = Path('/Users/ChristopherRivera/Desktop')
ab = desktop/'test.pdb'
ngl.show_structure_file(str(ab))

NGLWidget()

##### Code to renumber antibodies
One of the issues with AbBuilder is that it renumbers antibodies which can cause a problem downstream. Below is code to renumber them. 

In [14]:
df = pd.read_csv(ab, delim_whitespace=True, skiprows = 2, header = None)

In [5]:
numbers = df.drop_duplicates(subset=[4, 5])[5].reset_index(drop=True)

In [6]:
def renumber_pdb(input_pdb, output_pdb=None):
    """Renumbers residues for pdb file
    Args: 
        input_pdb(str|Path): path to input pdb
        output_pdb(str|Path): path to pdb for output
    Returns: 
        None
    """

    # parse the pdb and update numbers for each chain
    parser = PDBParser()
    struct= parser.get_structure('pdb', str(input_pdb))
    for model in struct: 
        for chain in model:
            num = 1
            for residue in chain:
                residue.id = (' ', num, ' ')
                num+=1

    # save the pdb
    if not output_pdb: 
        output_pdb = Path(input_pdb).with_suffix('.renumbered.pdb')
    pdb_io = PDBIO()
    pdb_io.set_structure(struct)
    pdb_io.save(str(output_pdb))
    
    

In [7]:
renumber_pdb(ab)

In [8]:
ab2 =str(desktop/ 'test.renumbered.pdb')
ngl.show_structure_file(ab2)

NGLWidget()

### Function for predicting pdb 
This is just a simple wrapper arround ImmuneBuilder AbodyBuilder2.

In [9]:
def predict_antibody_structures(sequences, output_dir=None):
    """Given a dataframe with sequences, use ABodyBuilder2 model to predict sequences
    Args: 
        sequence(pd.DataFrame|str|Path): data frame with columns 'Name', 'VH', 'VL'
        output_dir(path|Str): location to save models. 
    Returns: 
        None
    """

    predictor = ABodyBuilder2()
    if isinstance(sequences, str) or isinstance(sequences, Path):
        sequences = pd.read_csv(sequences)
    elif isinstance(sequences, pd.DataFrame): 
        pass
    else:
        print('Not antibodies')

    sequences= sequences.dropna()
    errors=[]

    iterator = zip(sequences['Name'], sequences['VH'], sequences['VL'])
    n = len(sequences)
    for name, hc, lc in tqdm(iterator, total=n):
        sequences = {'H': hc,'L': lc}
        try:
            antibody = predictor.predict(sequences)
            
            if output_dir: 
                output_file = output_dir/f'{name}.pdb'
                antibody.save(str(output_file))
        except AssertionError:
            errors.append(name)
  

In [10]:
abs = desktop/'therapeutic_antibodies.csv'
abs = pd.read_csv(abs).tail(628-166-217)

In [11]:
output_dir = desktop / 'ab_structures'
if not output_dir.exists(): 
    output_dir.mkdir()
predict_antibody_structures(abs, output_dir)

  0%|          | 0/456 [00:00<?, ?it/s]

Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context le

### Comment: 
As we can see it worked pretty well. 